<a href="https://colab.research.google.com/github/anshupandey/Generative-AI-for-Professionals/blob/main/OpenAI_External_API_Integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai --quiet
!pip install ipython --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.2 MB/s eta 0:00:00


In an API call, you can describe functions and have the model intelligently choose to output a JSON object containing arguments to call one or many functions. The Chat Completions API does not call the function; instead, the model generates JSON that you can use to call the function in your code.

## Supported models
Not all model versions are trained with function calling data. Function calling is supported with the following models: gpt-4, gpt-4-turbo-preview, gpt-4-0125-preview, gpt-4-1106-preview, gpt-4-0613, gpt-3.5-turbo, gpt-3.5-turbo-0125, gpt-3.5-turbo-1106, and gpt-3.5-turbo-0613

In addition, parallel function calls is supported on the following models: gpt-4-turbo-preview, gpt-4-0125-preview, gpt-4-1106-preview, gpt-3.5-turbo-0125, and gpt-3.5-turbo-1106

In [2]:
import os
import requests
import json
from openai import OpenAI
os.environ['OPENAI_API_KEY'] = "sk-xxxxxxxx"

# Initializing a client object: gets the API Key from environment variable OPENAI_API_KEY
client = OpenAI()

In [3]:
owm_api = "29af1cea50a401d8e624eea4660b3f59"

# create a dummy function to respond temperature
def get_current_weather(location, unit="fahrenheit"):
  "This function can be used to fetch the current weather information when provided with details such as location and unit for temperature"

  url = f"https://api.openweathermap.org/data/2.5/weather?q={location}&appid={owm_api}"
  response = requests.get(url)
  temp = response.json()['main']['temp']
  forecast = [response.json()['weather'][0]['main'],response.json()['weather'][0]['description']]

  weather_info = {
      "location":location,
      "temperature":temp,
      "unit":'Kelvin',
      "forecast":forecast
  }
  return json.dumps(weather_info)

In [6]:
get_current_weather("Manila")

'{"location": "Manila", "temperature": 303.37, "unit": "Kelvin", "forecast": ["Clouds", "few clouds"]}'

In [8]:
messages = [{"role":"user",'content':"what is AI?"}]
results = client.chat.completions.create(model = "gpt-3.5-turbo", messages = messages)
results.choices[0].message

ChatCompletionMessage(content='AI stands for Artificial Intelligence. It refers to the development of computer systems or machines that can perform tasks that would normally require human intelligence. AI involves the creation of algorithms and models to mimic human cognitive functions such as problem-solving, learning, perception, and decision-making. It encompasses a wide range of technologies, including machine learning, natural language processing, computer vision, and robotics, among others. AI enables computers to analyze large amounts of data, recognize patterns, make predictions, and adapt to changing circumstances, leading to applications in various fields such as healthcare, finance, autonomous vehicles, and virtual assistants.', role='assistant', function_call=None, tool_calls=None)

In [12]:
tools = [{
                            "type":"function",
                            "function":{"name": "get_current_weather",
                           "description": "This function can be used to fetch the current weather information when provided with details such as location and unit for temperature",
                           "parameters": {
                               "type": "object",
                               "properties": {"location": {
                                   "type": "string",
                                   "description": "The name of city/state/country for which weather information is to be fetched",},},
                               "required": ["location",],},},}]



def get_response(messages, tools,model="gpt-3.5-turbo-1106"):
  response = client.chat.completions.create(model = model, messages = messages, tools=tools, tool_choice='auto', temperature=0.5,)
  response = response.choices[0].message
  tool_calls = response.tool_calls

  try:
    if tool_calls:
      print("Making a function call")
      # get available functions
      available_functions = {
          "get_current_weather":get_current_weather,
      }
      print(tool_calls)

      # get details to call the function assuming there is only one function call
      func_name = tool_calls[0].function.name
      func_to_call = available_functions[func_name]
      func_args = json.loads(tool_calls[0].function.arguments)

      # call the external api by calling the function
      func_response = func_to_call(**func_args)

      # again make a call to openai api to communicate results from external function/API
      messages.append(response)
      messages.append(
          {"tool_call_id":tool_calls[0].id,
          "role":"tool",
          "name":func_name,
          "content":str(func_response)}
      )

      second_response = client.chat.completions.create(
      model = "gpt-3.5-turbo-1106",
      messages = messages)
      return second_response
    else:
      return response.content
  except Exception as e:
      print("Error occurred ",e)
      return response




In [13]:
messages = [{"role": "user", "content": "Provide a 2 line explanation for AI"}]
response = get_response(messages, tools)
print(response)

AI (Artificial Intelligence) refers to the simulation of human intelligence in machines that are programmed to think and act like humans. It encompasses a wide range of technologies and applications, including machine learning, natural language processing, and robotics.


In [14]:
messages = [{"role": "user", "content": "How is the weather in Mumbai today?"}]
response = get_response(messages, tools)
print(response.choices[0].message.content)

Making a function call
[ChatCompletionMessageToolCall(id='call_RD2DF2t561SY5QreaYNWvIzo', function=Function(arguments='{"location":"Mumbai"}', name='get_current_weather'), type='function')]
The weather in Mumbai today is hot with a temperature of 304.14 Kelvin and smoky conditions.
